In [1]:
## script de récupération des données TMF et GFC sur GEE

import yaml

def charger_fichier_yaml(nom_fichier):
    with open(nom_fichier, 'r') as fichier:
        contenu = fichier.read()
        donnees = yaml.safe_load(contenu)
    return donnees

# Exemple d'utilisation
nom_fichier_yaml = 'N:/Informatique/SIG/Application/Jupyterhub/projets/catalogFiles/data_config_files/TMF_annualChangeCollection_v12022_Dec_2021.yaml'
specs = charger_fichier_yaml(nom_fichier_yaml)
print(specs)

{'adaptingDataframe': {'changeType': {}, 'renameMap': {}, 'setValue': {'colName': 'id_indicateur', 'value': '??'}, 'toDrop': []}, 'bandName': '', 'catalogUri': 'None', 'confDb': {'chunksize': 1000, 'schema': 'surfor', 'strategy': 'append', 'tableName': 'faits_TMF_annualChangeCollection_v12022'}, 'confDims': {'isin_id_mesure': [3, 100], 'isin_id_spatial': ['None']}, 'confRaster': {'api': 'GEE', 'bandName': 'Dec2021', 'defaultValue': 255, 'masque': [1, 2, 3, 4, 5], 'outputValue': 'classifie', 'overlayHow': 'intersection', 'uri_image': 'projects/JRC/TMF/v1_2022/AnnualChanges/1ASIA'}, 'confTemporal': {'colName': 'annee', 'value': 2021, 'valuefromPixel': False}, 'dataName': 'TMF_annualChangeCollection_v12022_DEC_2021', 'epsg': 'EPSG:4326', 'keepList': ['classe', 'annee'], 'overlayHow': 'intersection', 'sourceType': 'Raster'}


In [2]:
%reload_ext autoreload
%autoreload 2
import ee
import os
import geemap
import sys
os.chdir('N:/Informatique/SIG/Etudes/2023/2309_QC_feux/Travail/Scripts/dynamic_land_cover/')
import gee as gee
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show
import pandas as pd
from shapely.geometry import Point
import numpy as np

chemin_output_csv='N:/Informatique/SIG/Etudes/2023/2309_PI_protocole_Surfor_CEO/OUTPUT_TMF/Foret_non_Foret_2021/'
data=pd.read_csv('N:/Informatique/SIG/Etudes/2023/2309_PI_protocole_Surfor_CEO/Scripts/Recup_TMF_GFC/point_foret_non_foret_2021_pixel.csv',delimiter=';')
file_output='TMF_foret_non_foret_2021.csv'

gee Imported


In [3]:
ee.Initialize()

geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]

gdf = gpd.GeoDataFrame(data, geometry=geometry)
gdf.set_crs(epsg=4326, inplace=True)

### check gdf length, cela créer le nombre nécessaire de gdf tout en gardant la condition que la longueur du gdf ne doit pas dépasser 5000 lignes
if len(gdf) > 5000:
    num_chunks = int(np.ceil(len(gdf) / 5000))
    gdfs = []
    
    for i in range(num_chunks):
        start_idx = i * 5000
        end_idx = start_idx + 5000
        gdfs.append(gdf.iloc[start_idx:end_idx])
else:
    gdfs = [gdf]

In [4]:
output=[]
for i, gdf in enumerate(gdfs, start=1):
    # Appliquer la commande d'extraction des données
    output_gdf = gee.extract_data(specs, gdf)
    print(output_gdf)

    output.append(output_gdf)
output_results=pd.concat(output,axis=0)
output_results.to_csv(chemin_output_csv+file_output)

Geometry type : Point or MultiPoint
     geometry  Dec2021        lat         lon  plotid  sampleid
0        None        1 -21.345649  165.286531       1         1
1        None        1 -21.345380  165.286531       1         2
2        None        1 -21.345111  165.286531       1         3
3        None        1 -21.345649  165.286820       1         4
4        None        1 -21.345380  165.286820       1         5
...       ...      ...        ...         ...     ...       ...
4995     None        1 -22.186619  166.930179     556      8881
4996     None        1 -22.186350  166.930179     556      8882
4997     None        1 -22.186081  166.930179     556      8883
4998     None        1 -22.186619  166.930470     556      8884
4999     None        1 -22.186350  166.930470     556      8885

[5000 rows x 6 columns]
Geometry type : Point or MultiPoint
     geometry  Dec2021        lat         lon  plotid  sampleid
0        None        1 -22.186081  166.930470     556      8886
1      